In [19]:
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO
from urllib.request import urlopen
import time

In [30]:
def getTables(response):
    # HTTPステータスコードが200（成功）の場合のみ処理を続行
    if response.getcode() == 200:
        # HTMLをパースしてBeautifulSoupオブジェクトを作成
        bs = BeautifulSoup(response, 'html.parser')
        #bs = bytes(bs,'UTF-8')
        bs = bs.decode('UTF-8')
        html_string = str(bs)
        html_io = StringIO(html_string)

        # テーブルデータを抽出
        tables = pd.read_html(html_io)
        df = tables[0]
        print(df)

    else:
        print(f"HTTPステータスコード {response.getcode()}: ページの取得に失敗しました")

   着  順  枠  馬  番        馬名  性齢    斤量   騎手     タイム     着差  人  気  単勝  オッズ   後3F  \
0     1  3     3   クイックバイオ  牝2  55.0   川田  1:29.9    NaN     1      2.2  35.5   
1     2  7     7   サクセスカノン  牝2  55.0   池添  1:30.3      2     2      4.4  35.8   
2     3  8     9    アイスノート  牝2  55.0  岩田望  1:30.6  1.3/4     4      7.1  36.1   
3     4  2     2  オレンジダンサー  牝2  55.0  西村淳  1:31.0  2.1/2     3      4.9  36.4   
4     5  4     4  セイウンマカロン  牝2  52.0  ▲佐藤  1:31.1    3/4     7     27.7  36.4   
5     6  6     6    カリンテラス  牝2  55.0  鮫島駿  1:31.4      2     5     12.2  36.5   
6     7  1     1    サノノリッチ  牝2  55.0  バデル  1:31.6  1.1/4     9    175.3  36.6   
7     8  5     5   ビバヴェローナ  牝2  55.0  Ｊ．モ  1:31.8  1.1/4     6     21.8  36.9   
8     9  8     8    フォカッチャ  牝2  55.0   丹内  1:32.7      5     8    148.8  37.5   

  コーナー  通過順       厩舎   馬体重  (増減)  
0     3-1-1   栗東  須貝   488  (-8)  
1     6-2-2   栗東  北出   422  (-4)  
2     1-2-2  栗東  高橋忠  452  (-10)  
3     3-4-4   栗東  高野   422  (+4)  
4     2-4-5  美

In [ ]:
url = 'https://race.netkeiba.com/race/result.html?race_id='
response = urlopen(url)
getTables(response)